# Portfolio Optimization using Second Order Cone

In this notebook we show how to use the Second Order Cone (SOC) constraint in the variance portfolio optimization problem.

## 1. Variance Optimization

### 1.1 Variance Minimization

The minimization of portfolio variance is a quadratic optimization problem that can be posed as:

$$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{min}} & &  x^{\tau} \Sigma x \\
& \text{s.t.} & & \mu x^{\tau} \geq \bar{\mu} \\
& & &  \sum_{i=1}^{N} x_i = 1 \\
& & &  x_i \geq 0 \; ; \; \forall \; i =1, \ldots, N \\
\end{aligned}
\end{equation}
$$

Where $x$ are the weights of assets, $\mu$ is the mean vector of expected returns and $\bar{\mu}$ the minimum expected return of portfolio.

In [1]:
####################################
# Downloading Data
####################################

import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")

yf.pdr_override()
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2016-01-01'
end = '2019-12-30'

# Tickers of assets
assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'MMC', 'JPM',
          'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 'PCAR', 'TXT', 'TMO',
          'DE', 'MSFT', 'HPQ', 'SEE', 'VZ', 'CNP', 'NI', 'T', 'BA']
assets.sort()

# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets

# Calculating returns
Y = data[assets].pct_change().dropna()

display(Y.head())

[*********************100%***********************]  25 of 25 completed


,APA,BA,BAX,BMY,CMCSA,CNP,CPB,DE,HPQ,JCI,...,NI,PCAR,PSA,SEE,T,TGT,TMO,TXT,VZ,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,-2.0256%,0.4057%,0.4036%,1.9693%,0.0180%,0.9304%,0.3678%,0.5783%,0.9483%,-1.1953%,...,1.5882%,0.0212%,2.8236%,0.9758%,0.6987%,1.7539%,-0.1730%,0.2410%,1.3734%,-1.0857%
2016-01-06,-11.4863%,-1.5879%,0.2412%,-1.7557%,-0.7727%,-1.2473%,-0.1736%,-1.1238%,-3.5867%,-0.9551%,...,0.5547%,0.0212%,0.1592%,-1.5647%,-0.1466%,-1.0155%,-0.7653%,-3.0048%,-0.9034%,-2.9145%
2016-01-07,-5.1389%,-4.1922%,-1.6573%,-2.7699%,-1.1047%,-1.9769%,-1.2206%,-0.8856%,-4.6058%,-2.5394%,...,-2.2066%,-3.0309%,-1.0410%,-3.1557%,-1.6148%,-0.2700%,-2.2845%,-2.0570%,-0.5492%,-3.0020%
2016-01-08,0.2736%,-2.2705%,-1.6037%,-2.5425%,0.1099%,-0.2241%,0.5706%,-1.6403%,-1.7642%,-0.1649%,...,-0.1538%,-1.1366%,-0.7308%,-0.1448%,0.0895%,-3.3839%,-0.1117%,-1.1387%,-0.9719%,-1.1254%
2016-01-11,-4.3383%,0.1692%,-1.6851%,-1.0215%,0.0915%,-1.1791%,0.5674%,0.5288%,0.6617%,0.0331%,...,1.6435%,0.0000%,0.9869%,-0.1450%,1.2224%,1.4570%,0.5367%,-0.4607%,0.5800%,-1.9919%


In [2]:
####################################
# Minimizing Portfolio Variance
####################################

import cvxpy as cp
from timeit import default_timer as timer

# Defining initial inputs
mu = Y.mean().to_numpy().reshape(1,-1)
sigma = Y.cov().to_numpy()

# Defining initial variables
x = cp.Variable((mu.shape[1], 1))

# Budget and weights constraints
constraints = [cp.sum(x) == 1,
               x <= 1,
               x >= 0]

# Defining risk objective
risk = cp.quad_form(x, sigma)
objective = cp.Minimize(risk)

weights = pd.DataFrame([])
# Solving the problem with several solvers
prob = cp.Problem(objective, constraints)
solvers = ['ECOS', 'SCS']
for i in solvers:
    prob.solve(solver=i)
    # Showing Optimal Weights
    weights_1 = pd.DataFrame(x.value, index=assets, columns=[i])
    weights = pd.concat([weights, weights_1], axis=1)

display(weights)

,ECOS,SCS
APA,0.0002%,-0.0002%
BA,0.0012%,-0.0015%
BAX,5.2647%,5.2629%
BMY,4.3893%,4.3889%
CMCSA,2.1705%,2.1700%
CNP,6.9880%,6.9875%
CPB,3.2389%,3.2391%
DE,0.0708%,0.0797%
HPQ,0.0001%,-0.0010%
JCI,2.8381%,2.8376%


As we can see the use of CVXPY's __quad_form__ in portfolio optimization can give small negative values to weights that must be zero.

In [3]:
# Calculating Annualized Portfolio Stats
var = weights * (Y.cov() @ weights) * 252
var = var.sum().to_frame().T
std = np.sqrt(var)
ret = Y.mean().to_frame().T @ weights * 252

stats = pd.concat([ret, std, var], axis=0)
stats.index = ['Return', 'Std. Dev.', 'Variance']

display(stats)

,ECOS,SCS
Return,12.8507%,12.8504%
Std. Dev.,10.3737%,10.3735%
Variance,1.0761%,1.0761%


### 1.2 Return Maximization with Variance Constraint

The maximization of portfolio return is a problem with a quadratic constraint that can be posed as:

$$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{max}} & & \mu x^{\tau} \\
& \text{s.t.} & & x^{\tau} \Sigma x \leq \bar{\sigma}^{2} \\
& & & \sum_{i=1}^{N} x_i = 1 \\
& & &  x_i \geq 0 \; ; \; \forall \; i =1, \ldots, N \\
\end{aligned}
\end{equation}
$$

Where $x$ are the weights of assets and $\bar{\sigma}$ is the maximum expected standard deviation of portfolio..

In [4]:
#########################################
# Maximizing Portfolio Return with
# Variance Constraint
#########################################

import cvxpy as cp
from timeit import default_timer as timer

# Defining initial inputs
mu = Y.mean().to_numpy().reshape(1,-1)
sigma = Y.cov().to_numpy()

# Defining initial variables
x = cp.Variable((mu.shape[1], 1))
sigma_hat = 15 / (252**0.5 * 100)
ret = mu @ x

# Budget and weights constraints
constraints = [cp.sum(x) == 1,
               x <= 1,
               x >= 0]

# Defining risk constraint and objective
risk = cp.quad_form(x, sigma)
constraints += [risk <= sigma_hat**2] # variance constraint
objective = cp.Maximize(ret)

weights = pd.DataFrame([])
# Solving the problem with several solvers
prob = cp.Problem(objective, constraints)
solvers = ['ECOS', 'SCS']
for i in solvers:
    prob.solve(solver=i)
    # Showing Optimal Weights
    weights_1 = pd.DataFrame(x.value, index=assets, columns=[i])
    weights = pd.concat([weights, weights_1], axis=1)

display(weights)

,ECOS,SCS
APA,0.0000%,0.0000%
BA,9.5892%,9.5898%
BAX,12.6175%,12.6257%
BMY,0.0000%,0.0063%
CMCSA,0.0000%,0.0021%
CNP,3.7021%,3.7317%
CPB,0.0000%,-0.0011%
DE,6.2565%,6.2573%
HPQ,0.0000%,-0.0026%
JCI,0.0000%,-0.0063%


The small negative values also appear when we use CVXPY's __quad_form__ in constraints.

In [5]:
# Calculating Annualized Portfolio Stats
var = weights * (Y.cov() @ weights) * 252
var = var.sum().to_frame().T
std = np.sqrt(var)
ret = Y.mean().to_frame().T @ weights * 252

stats = pd.concat([ret, std, var], axis=0)
stats.index = ['Return', 'Std. Dev.', 'Variance']

display(stats)

,ECOS,SCS
Return,26.0352%,26.0382%
Std. Dev.,15.0000%,15.0002%
Variance,2.2500%,2.2501%


## 2 Standard Deviation Optimization

### 2.1 Standard Deviation Minimization

An alternative problem is to minimize the standard deviation (square root of variance). To do this we can use the SOC constraint. The minimization of portfolio standard deviation can be posed as:

$$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{min}} & &  g \\
& \text{s.t.} & & \mu x^{\tau} \geq \bar{\mu} \\
& & &  \sum_{i=1}^{N} x_i = 1 \\
& & & \left\|\Sigma^{1/2} x\right\| \leq g \\
& & &  x_i \geq 0 \; ; \; \forall \; i =1, \ldots, N  \\
\end{aligned}
\end{equation}
$$

Where $\left\|\Sigma^{1/2} x\right\| \leq g$ is the SOC constraint, $x$ are the weights of assets, $\mu$ is the mean vector of expected returns, $\bar{\mu}$ the minimum expected return of portfolio and $r$ is the matrix of observed returns.

__Note:__ the SOC constraint can be expressed as $(g,\Sigma^{1/2} x) \in Q^{n+1}$, this notation is used to model the __SOC constraint__ in CVXPY.

In [6]:
#########################################
# Minimizing Portfolio Standard Deviation
#########################################

from scipy.linalg import sqrtm

# Defining initial inputs
mu = Y.mean().to_numpy().reshape(1,-1)
sigma = Y.cov().to_numpy()
G = sqrtm(sigma)

# Defining initial variables
x = cp.Variable((mu.shape[1], 1))
g = cp.Variable(nonneg=True)

# Budget and weights constraints
constraints = [cp.sum(x) == 1,
               x >= 0]

# Defining risk objective
risk = g
constraints += [cp.SOC(g, G @ x)] # SOC constraint
constraints += [risk <= sigma_hat] # variance constraint
objective = cp.Minimize(risk)

weights = pd.DataFrame([])
# Solving the problem with several solvers
prob = cp.Problem(objective, constraints)
solvers = ['ECOS', 'SCS']
for i in solvers:
    prob.solve(solver=i)
    # Showing Optimal Weights
    weights_1 = pd.DataFrame(x.value, index=assets, columns=[i])
    weights = pd.concat([weights, weights_1], axis=1)

display(weights)

,ECOS,SCS
APA,0.0000%,-0.0000%
BA,0.0000%,0.0000%
BAX,5.2634%,5.2634%
BMY,4.3887%,4.3887%
CMCSA,2.1705%,2.1705%
CNP,6.9870%,6.9870%
CPB,3.2391%,3.2391%
DE,0.0790%,0.0791%
HPQ,0.0000%,0.0000%
JCI,2.8376%,2.8376%


As we can see the use of CVXPY's __SOC constraint__ in portfolio optimization solves the error that we see when we use __quad_form__.

In [7]:
# Calculating Annualized Portfolio Stats
var = weights * (Y.cov() @ weights) * 252
var = var.sum().to_frame().T
std = np.sqrt(var)
ret = Y.mean().to_frame().T @ weights * 252

stats = pd.concat([ret, std, var], axis=0)
stats.index = ['Return', 'Std. Dev.', 'Variance']

display(stats)

,ECOS,SCS
Return,12.8508%,12.8507%
Std. Dev.,10.3737%,10.3737%
Variance,1.0761%,1.0761%


### 2.2 Return Maximization with Standard Deviation Constraint

The maximization of portfolio return using SOC constraints can be posed as:

$$
\begin{equation}
\begin{aligned}
& \underset{x}{\text{max}} & & \mu x^{\tau} \\
& \text{s.t.} & & g \leq \bar{\sigma} \\
& & & \left\|\Sigma^{1/2} x\right\| \leq g \\
& & & \sum_{i=1}^{N} x_i = 1 \\
& & &  x_i \geq 0 \; ; \; \forall \; i =1, \ldots, N \\
\end{aligned}
\end{equation}
$$

In [8]:
#########################################
# Maximizing Portfolio Return with
# Standard Deviation Constraint
#########################################

from scipy.linalg import sqrtm

# Defining initial inputs
mu = Y.mean().to_numpy().reshape(1,-1)
sigma = Y.cov().to_numpy()
G = sqrtm(sigma)

# Defining initial variables
x = cp.Variable((mu.shape[1], 1))
g = cp.Variable(nonneg=True)
sigma_hat = 15 / (252**0.5 * 100)
ret = mu @ x

# Budget and weights constraints
constraints = [cp.sum(x) == 1,
               x <= 1,
               x >= 0]


# Defining risk constraint and objective
risk = g
constraints += [cp.SOC(g, G @ x)] # SOC constraint
constraints += [risk <= sigma_hat] # standard deviation constraint
objective = cp.Maximize(ret)

weights = pd.DataFrame([])
# Solving the problem with several solvers
prob = cp.Problem(objective, constraints)
solvers = ['ECOS', 'SCS']
for i in solvers:
    prob.solve(solver=i)
    # Showing Optimal Weights
    weights_1 = pd.DataFrame(x.value, index=assets, columns=[i])
    weights = pd.concat([weights, weights_1], axis=1)

display(weights)

,ECOS,SCS
APA,0.0000%,-0.0000%
BA,9.5886%,9.5882%
BAX,12.6189%,12.6204%
BMY,0.0000%,-0.0000%
CMCSA,0.0000%,-0.0000%
CNP,3.7154%,3.7270%
CPB,0.0000%,-0.0000%
DE,6.2556%,6.2547%
HPQ,0.0000%,-0.0000%
JCI,0.0000%,0.0000%


CVXPY's __SOC constraint__ also solves the error that we see when we use __quad_form__ in constraints.

In [9]:
# Calculating Annualized Portfolio Stats
var = weights * (Y.cov() @ weights) * 252
var = var.sum().to_frame().T
std = np.sqrt(var)
ret = Y.mean().to_frame().T @ weights * 252

stats = pd.concat([ret, std, var], axis=0)
stats.index = ['Return', 'Std. Dev.', 'Variance']

display(stats)

,ECOS,SCS
Return,26.0352%,26.0353%
Std. Dev.,15.0000%,15.0001%
Variance,2.2500%,2.2500%


For more portfolio optimization models and applications, you can see the CVXPY based library __[Riskfolio-Lib](https://github.com/dcajasn/Riskfolio-Lib)__.

Finally, I hope you liked this example.